# Final notebook?

In [ ]:
import numpy as np
import math as math
import matplotlib.pyplot as plt
import statistics as stats
from mpl_toolkits.mplot3d import Axes3D
from itertools import product
from numba import jit
from matplotlib import rc


# Define font for figures
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

import time
%matplotlib inline

## Initalizing algorithm 

In [ ]:
def position(l,N,L):
    """
    Initializes position based on density. Returns r (N x 3) matrix with x-y-z position of particles
    l is unit cell length 
    N is amount of particles
    L is number of unit cells in each direction
    """
    # particles in unit cell
    r = np.zeros(shape=(N, 3), dtype="float64")
    #coordinates of 4 particles in the unit cell
    p1 = l * np.array([0.25, 0.25, 0.25])
    p2 = l*np.array([0.75, 0.75, 0.25])
    p3 = l*np.array([0.75, 0.25, 0.75])
    p4 = l*np.array([0.25, 0.75, 0.75])

    # distribute all the particles by using the unit cell and displacing it in x y and z with length l
    n=0
    for x, y, z in product(range(L), range(L), range(L)):
        disp = np.multiply(l, np.array([x, y, z])) #displacement array
        r[n] = p1 + disp
        r[n + 1] = p2 + disp
        r[n + 2] = p3 + disp
        r[n + 3] = p4 + disp
        n += 4
        
    return r

def velocity(T,N):
    """
    Initializes velocity based on Maxwell distribution of speed and makes sure avg momentum is 0
    T is temperature
    N is amount of particles
    Returns v (N x 3) with vx-vy-vz
    """
    v = np.zeros(shape=(N, 3), dtype="float64")
    sigma = math.sqrt(T) #variance of the system
    mu = 0 #mean speed
    v = np.random.normal(mu, sigma, 3*N).reshape(-1, 3)
    v -= v.sum(axis=0) / N
    return v

## Force algorithm

In [ ]:
@jit
def Forces(r, ld, N, bins, binlen):
    
    """
    compute forces on all particles.
    r - Nx3 matrix containing x y z positions of all particles
    ld - length of the computional domain (for a box of Volume = ld**3)
    N - amount of particles
    bins - amount of bins for the correlation function
    binlen-length of the bin:ld/bins
    Returns force on particles (acc), amount of particles in shell (n), virial component (pressure), and Potential (V)
    """
    #initiate vectors and 
    n=np.zeros(shape=(bins,))
    acc=np.zeros(shape=(N,3))
    pressure=0
    V = 0
    
    for i in range(N):
        for j in range(i):
            dx=r[i,0]-r[j,0]
            dy=r[i,1]-r[j,1]
            dz=r[i,2]-r[j,2]
            dx -= np.rint(dx / ld) * ld
            dy -= np.rint(dy / ld) * ld
            dz -= np.rint(dz / ld) * ld
            dr2= dx * dx + dy * dy + dz * dz
            drreturn = dr2 #remember dr2 to use for pressure and n
            force=0
             
            if dr2 < rcutoff:
                dr2 = 1 / dr2
                dr6 = dr2 * dr2 * dr2
                dr12 = dr6 * dr6
                dr14 = dr12 * dr2
                dr8 = dr6 * dr2
                V += 4 * (dr12 - dr6)
                force = 24 * (2 * dr14 - dr8)
                acc[i,0] += force * dx
                acc[i,1] += force * dy
                acc[i,2] += force * dz
                acc[j,0] -= force * dx
                acc[j,1] -= force * dy
                acc[j,2] -= force * dz
            distances=np.sqrt(drreturn)
            n[int(distances/binlen)] +=1 #distribution for correlation function
            pressure += drreturn * (-1 * force) #virial component to calculate pressure
            
            
    return acc,n,pressure,V

## THE Algorithm

In [ ]:
def Mainalgorithm(l,N,L,T,ld,bins,binlen,Time,lt):
    #initiate n matrix
    n = np.zeros(shape=(bins,Time-lt))
    V = np.zeros(shape=(Time,))

    #initiate atoms
    r = position(l,N,L)
    v = velocity(T,N)
    F, n[:,0], pressure, Vn = Forces(r, ld, N,bins,binlen)

    #initiate vectors 
    pressure = 0
    K = np.zeros(shape=(Time,))
    E = np.zeros(shape=(Time,))
    Pi = np.zeros(shape=(Time-lt,))
    Tr = np.zeros(shape=(Time-lt,))


    for i in range(Time):
        ## Verlet Algorithm
        v += 0.5* F * dt #halfway step for velocity (verlet)
        r += v * dt #update position
        r = np.mod(r , (ld)) #periodic boundary
        F,nt,pressuret,Vt = Forces(r,ld,N,bins,binlen) #forces due to new position
        v += 0.5 * F * dt #complete velocity step
        K[i] = 0.5*np.sum( v * v)
        V[i] = Vt
        E[i] = K[i] + V[i]
    
        #Thermostat
        if i<lt and (np.mod(i,10)==0): #mod for calling thermostat 10 times in time lt
            Ktherm = 0.5*np.sum( v * v)
            scale=np.sqrt(1.5*(N-1)*T/Ktherm)
            v=scale*v
            K[i] = 0.5*np.sum(v*v)
            E[i] = K[i] + V[i]
        
        if i>lt: 
            #Only do pressure when not using thermostat
            Pi[i-lt] = pressuret
            n[:,i-lt] = nt
            Tr[i-lt] = 2*K[i]/(3*(N-3))
    
    return K, E, Pi, n, Tr, V

## Choose constants and initial conditions

In [ ]:
## get main algorithm results

## preferrably unvariable
L = 6 #number of unit cells in 3 directions
Z = 4 #number of atoms per unit cell
N = Z*L**3 #number of atoms in total space
rcutoff = 9 #sigma ^2 cut off range

## input parameters , for controlling the phase 

## Uncomment for useful points in pressure
#rho = [0.1,0.2,0.3,0.4,0.5,0.6,0.65,0.7,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.65,0.68,0.72,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.65]#density of particles
#T = [1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.35,1.35,1.35,1.35,1.35,1.35,1.35,1.35,1.35,1.35,2.74,2.74,2.74,2.74,2.74,2.74,2.74,2.74] #Temperature in reduced units

## Uncomment for gas and liquid
#rho = [0.01,0.6775]
#T = [5,0.7833]

## Uncomment for a solid (many more timesteps have to be taken to get reliable results for Cv!)
#rho = [0.88,0.88]
#T = [0.01,0.01]


## Timestep algorithm constants
Time = 15000 #amount of timesteps 
lt = 2500 #amount of timesteps to thermostate
dt = 0.004 #length of timestep

# ## correlation function variables
bins = 500


## Global matrices
Kf = np.zeros(shape=(Time,len(rho)))
Ef  = np.zeros(shape=(Time,len(rho)))
Pif = np.zeros(shape=(Time-lt,len(rho)))
nf = np.zeros(shape=(bins,Time-lt,len(rho)))
pressure = np.zeros(shape = (len(rho),))
Tr = np.zeros(shape=(Time-lt,len(rho)))
Vf = np.zeros(shape=(Time,len(rho)))


## Run algorithm

In [ ]:
#This algorithm runs the simulation and saves the main properties from the simulation. 
"""
The main properties saved for each timestep and are: Kinetic energy, Total eenrgy, Virial term, distribution n (in bins)
Temperature and potential energy.
"""


for i in range(len(rho)):
    ## input parameters , for controlling the phase 
    rhoi = rho[i]

    ## parameters that are influenced by input parameters
    l = (Z/rhoi)**(1/3) #unit cell length in sigma
    ld = L*l #computational domain length
    
    ## correlation function variables
    binlen = ld/bins 
    
    
    Kf[:,i], Ef[:,i], Pif[:,i], nf[:,:,i], Tr[:,i], Vf[:,i] = Mainalgorithm(l,N,L,T[i],ld,bins,binlen,Time,lt)

np.save('Kinetic_1', Kf)
np.save('Total_1',Ef)
np.save('Pressure_1',Pif)
np.save('histbins_1',nf)
np.save('Temperature_1',Tr)   

#It is recommended to keep an extra file with information on the data in this string
    

In [ ]:
## Create a figure to show constant energy. len(rho) has to be 1 for this one!

fig = plt.figure(figsize=(2.5,2.5))
ax = fig.add_subplot(111)
ax.plot(Kf, 'k--',label = 'Kinetic')
ax.plot(Ef,'k-',label = 'Total')
ax.plot(Vf,'k-.',label= 'Potential')
plt.xlabel(r'timestep',fontsize = 9)
plt.ylabel(r'Energy ($\epsilon$)')
lgd = ax.legend(loc=9, bbox_to_anchor=(0.5, -0.11), ncol=3, fontsize = 10)
art = []
art.append(lgd)


In [ ]:
# To save the figure

fig.savefig("energy_const.pdf", additional_artists=art,bbox_inches="tight")


## Cv

In [ ]:
# Computation of Cv and the error

Cv = np.zeros(shape=(len(rho),))
Cverr = np.zeros(shape=(len(rho),))

intervals = 51

for j in range(len(rho)):
    Knew = Kf[lt:,j] #Take data from after the thermostat is done
    blocks=np.linspace(0,len(Knew),intervals) #Dividing the Energy in equal intervals
    dblocks=np.diff(blocks)# Length of the interval
    dbl = int(dblocks[1]) #length of each interval

    Cvblock=np.zeros(shape=(len(blocks),)) # Initialization of the heat capacity vector

    # Computation of the errors by computing Cv in each interval and taking the error of these computations
    for i in range(len(blocks)-1):
        Kerr=Knew[i*dbl:dbl+i*dbl] # Taking just the elements of each interval
        #Cv for each block
        Kmean = np.mean(Kerr)
        Kmean2 = Kmean*Kmean 
        Kvar = np.var(Kerr)
        Cvblock[i] = (3*Kmean2)/(2*Kmean2 - 3*N*Kvar)

    Cverr[j]= np.std(Cvblock)/np.sqrt(len(dblocks))#Cvstd/(len(blocks)*math.sqrt(len(blocks)))
    
    #Calculate Cv from the whole ensemble
    Kmean = np.mean(Knew)
    Kmean2 = Kmean*Kmean 
    Kvar = np.var(Knew)
    Cv[j] = (3*Kmean2)/(2*Kmean2 - 3*N*Kvar) # Cv /N (to check Cv/N = 1.5 kb for hot gas and Cv/N = 3 for solid)
    print("Cv is", Cv[j], "+-", Cverr[j])

## Correlation fuction

In [ ]:
# #Uncomment for solid
# # nf = np.load('histbins_L6_T100000_solid.npy')
# # #Calculate Correlation function
# # Time = 200000 #amount of timesteps 
# # lt = 50000 #amount of timesteps to thermostate
# # T = [0.01]
# # rho = [0.88]

# #Uncomment for liquid
# nf = np.load('histbins_L6_T12500_liquid.npy')
# #Calculate Correlation function
# #Time = 200000 #amount of timesteps 
# #lt = 50000 #amount of timesteps to thermostate
# T = [0.7833]
# rho = [0.6775]


intervals = 21

blocks = np.linspace(0,len(nf[1,:,0]),intervals)
dblocks = np.diff(blocks)
dbl = int(dblocks[1])

nerrblock = np.zeros(shape=(bins,dbl))
ncorrblock = np.zeros(shape=(bins,len(dblocks)))
Vshell = np.zeros(shape=(len(rho),bins))
nerr = np.zeros(shape=(bins,len(rho)))

for i in range(len(rho)):
    l = (Z/rho[i])**(1/3) #unit cell length in sigma
    ld = L*l #computational domain length
    binlen = ld/bins 
    for j in range(bins):
        Vshell[i,j]=4*math.pi*(((j+0.5)*binlen)**2)*binlen #using r = (i+0.5)*dr for "average" distance for bin, 
        #computing the volume of the shell, binlen and ld depend on rho

for j in range(len(rho)):
    # Computation of the errors by computing corr for each block and taking the standard error in these computations
    #for the error
    for i in range(len(blocks)-1):
        nerrblock = nf[:,i*dbl:(i+1)*dbl,j] # Taking just the elements of each interval
        navg = np.sum(nerrblock,axis=1)/dbl
        ncorrblock[:,i] = 2/(rho[j]*(N-1))* navg / Vshell[j,:]
        
        #err in each bin for each value of rho and T
    for i in range(bins):
        nblock = ncorrblock[i,:]
        nblockstd = np.std(nblock)
        nerr[i,j] = nblockstd/np.sqrt(len(dblocks))
    
for i in range(len(rho)):
    l = (Z/rho[i])**(1/3) #unit cell length in sigma
    ld = L*l #computational domain length
    navg = np.average(nf[:,:,i],axis = 1)
    ncorr = navg/Vshell[i,:]
    
    corr=2/(rho[i]*(N-1))*ncorr #correlation function
    xaxis = np.linspace(0,ld,bins) #xaxis for correlation function
    lower_bound = corr - nerr[:,i]
    upper_bound = corr + nerr[:,i]



    #Plotting of the correlation function
    fig = plt.figure(figsize=(3.5,5.66))
    ax2 = fig.add_subplot(111)
    ax2.plot(xaxis,corr,'k-',linewidth = 1)
    ax2.plot(xaxis,corr,'ko',markersize = 1)
    #ax2.plot([1,1,1,1,1,1,1],'k',linewidth = 0.5) uncomment for thin line at 1
    ax2.set_xlim([0, 0.5*np.max(xaxis)])
    ax2.set_ylim([0, 1.2*np.max(corr)])
    plt.xlabel(r'$r/ \sigma$', fontsize=9)
    plt.ylabel(r'$g(r)$', fontsize=9)
    #shaded area to indicate error. 
    ax2.fill_between(xaxis,lower_bound,upper_bound, color = 'k',alpha=0.5)
    

In [ ]:
fig.savefig('correlationfunc_solid_8_axis.pdf', bbox_inches='tight', pad_inches=0.1)

## Pressure

In [ ]:
# The pressure and it's error. The error has been found to be negligible, and produces no nice results in the plot
# when using the errorbar function.
intervals = 3

#Vectors for error calculation
blocks = np.linspace(0,len(Pif[:,0]),intervals)
dblocks = np.diff(blocks)
dbl = int(dblocks[1])

P2err = np.zeros(shape=(len(rho),))
P2bins = np.zeros(shape=(len(dblocks),))

#cutoff integral for different rho and T
cofactor = [2*math.pi*rho[i]/(3*np.average(Tr[:,i])) * 0.2960253349 for i in range(len(rho))] 

#Error calculation by dividing into blocks, calculating pressure in each 
#block and taking the standard error of those blocks
for j in range(len(rho)):
    for i in range(len(dblocks)):
        Virerr = Pif[i*dbl:(i+1)*dbl,j] 
        avgvirial = (1/(3*N*np.average(Tr))) * (np.average(Virerr))
        P2bins[i] = 1 - avgvirial - cofactor[j]
    
    P2err[j] = np.std(P2bins)/np.sqrt(len(dblocks))


#Experimental data poitns from Levelt
dlevelt = [0.7244, 0.5270, 0.4045, 0.3684, 0.3493, 0.3527, 0.3882, 0.6218, 0.8633]
rholevelt = [0.1, 0.2, 0.3, 0.35,0.4,0.45, 0.5, 0.6,0.650]
#Levelt data in P/rho*k*T, converstion to P/kT
Plevelt = [rholevelt[i]*dlevelt[i] for i in range(len(dlevelt))]

avgvirial = np.zeros(shape=(len(rho),))
P2 = np.zeros(shape=(len(rho),))
#Calculation of all P/rho*k*T
for i in range(len(rho)):
    avgvirial[i] = (1/(3*N*np.average(Tr[:,i]))) * (np.sum(Pif[:,i],axis=0) / (Time-lt))
    P2[i] = 1 -  avgvirial[i] - cofactor[i]
# from P/rho*k*T to P/kT    
Pline = rho*P2



In [ ]:
# Make plot. Play around with rho to find the isotherms, as they are all in 1 vector (could be updated in next iteration)
# The isotherms are done here for the points in pressure

fig = plt.figure(figsize=(3.5,5.66))
ax2 = fig.add_subplot(111)
T125 = ax2.plot(rho[0:8], Pline[0:8], 'ok-', markersize = 3, label= '$T = 1.25$')
T135 = ax2.plot(rho[8:18], Pline[8:18],'xk-', markersize = 5, label='$T = 1.35$')
T274 = ax2.plot(rho[18:], Pline[18:],'^k-', markersize = 5, label = '$T = 2.74$')
Levelt = ax2.scatter(rholevelt,Plevelt, facecolors ='none', edgecolors='k', label ='experimental data')
plt.xlabel(r'$\rho~[\sigma ^{-3}]$', fontsize=9)
ylab = plt.ylabel(r'$\frac{P}{k T}~[\sigma^{-3}]$', fontsize=9)
ylab.set_rotation(0)
legend = ax2.legend(loc='upper left', shadow=True, fontsize = 9)
ax2.set_xlim([0, 0.9])
ax2.set_ylim([0, 2.5])

ax2.yaxis.set_label_coords(-0.2,0.6)

In [ ]:
fig.savefig('pressure_inflection_experiments.pdf', bbox_inches='tight', pad_inches=0.1)

## Temperature

In [ ]:
intervals = 3

blocks=np.linspace(0,len(Tr),intervals) #Dividing the Temperature in equal intervals
dblocks=np.diff(blocks)# Length of the interval
Tmean = np.zeros(shape=(len(dblocks),))
       
dbl = int(dblocks[1])
Tavg=0
Terr=0

for j in range(len(rho)):
    Tblock=np.zeros(shape=(len(blocks),)) # Initialization of the temperature vector

    # Computation of the errors
    for i in range(len(blocks)-1):
        Tblock=Tr[i*dbl:dbl+i*dbl,j] # Taking just the elements of each interval
        Tmean[i] = np.mean(Tblock)

    Terr = np.std(Tmean)/np.sqrt(len(blocks)-1)
    Tavg2 = np.average(Tr[:,j])
    print(Tavg2, "+-", Terr)